In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import differential_evolution
from gravity3d_variable_density import compute_gravity

Lx, Ly = 15000, 15000
nx, ny, nz = 60, 60, 50
max_depth1, max_depth2 = 1600, 1100

x_edges = np.linspace(0, Lx, nx + 1)
y_edges = np.linspace(0, Ly, ny + 1)
z_edges = np.linspace(0, max_depth1, nz + 1)
xc = 0.5 * (x_edges[:-1] + x_edges[1:])
yc = 0.5 * (y_edges[:-1] + y_edges[1:])
zc = 0.5 * (z_edges[:-1] + z_edges[1:])
Xc2d, Yc2d = np.meshgrid(xc, yc, indexing='ij')

x1, y1, Rmax1 = Lx * 0.3, Ly / 2, 3900
x2, y2, Rmax2 = Lx * 0.65, Ly / 2, 4500
R1 = np.sqrt((Xc2d - x1)**2 + (Yc2d - y1)**2)
R2 = np.sqrt((Xc2d - x2)**2 + (Yc2d - y2)**2)
basin1 = max_depth1 * (1 - (R1 / Rmax1)**2); basin1[R1 > Rmax1] = 0
basin2 = max_depth2 * (1 - (R2 / Rmax2)**2); basin2[R2 > Rmax2] = 0
basin_depth_true = np.maximum(basin1, basin2)

rho_top, rho_bottom = -550, -850
density_profile = np.linspace(rho_top, rho_bottom, nz)
density_contrast = np.zeros((nx, ny, nz))

for k in range(nz):
    z = zc[k]
    density_contrast[:, :, k] = (z <= basin_depth_true) * density_profile[k]


Xobs, Yobs = Xc2d.copy(), Yc2d.copy()
Zobs = np.zeros_like(Xobs)

gz_obs = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, density_contrast)




In [ ]:

# B-spline setup (coarse control points)
nx_c, ny_c = 8, 8
x_ctrl = np.linspace(0, Lx, nx_c)
y_ctrl = np.linspace(0, Ly, ny_c)
Xctrl, Yctrl = np.meshgrid(x_ctrl, y_ctrl, indexing='ij')

bounds = [(500, 2500)] * (nx_c * ny_c)

def spline_surface(params):
    ctrl_depths = params.reshape((nx_c, ny_c))
    spline = RectBivariateSpline(x_ctrl, y_ctrl, ctrl_depths, kx=3, ky=3)
    return spline(xc, yc)

def model_from_spline_surface(surf):
    model = np.zeros((nx, ny, nz))
    for k in range(nz):
        z = zc[k]
        mask = z <= surf
        model[:, :, k] = mask * density_profile[k]
    return model

def objective_function(params):
    surf = spline_surface(params)
    model = model_from_spline_surface(surf)
    gz_pred = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, model)
    misfit = np.linalg.norm(gz_pred - gz_obs) / np.linalg.norm(gz_obs)
    return misfit


result = differential_evolution(objective_function, bounds, maxiter=80, popsize=15, recombination=0.7, disp=True)
best_params = result.x

# Recovered model
recovered_surface = spline_surface(best_params)
recovered_density = model_from_spline_surface(recovered_surface)
gz_recovered = compute_gravity(Xobs, Yobs, Zobs, x_edges, y_edges, z_edges, recovered_density)
